In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import bgk
import bgk.run_params as rp

# Helpers

In [ ]:
def moving_average(arr: list[float], window: int) -> list[float]:
    ret = np.cumsum(arr)
    ret[window:] = ret[window:] - ret[:-window]
    return ret[window - 1:] / window

def plot(xs: list[float], *ys_argss: list):
    plt.close("all")
    for [ys, *args] in ys_argss:
        if callable(ys):
            ys = [ys(x) for x in xs]
        plt.plot(xs, ys, *args)

def print_errs(pnames: list[str], popts: np.ndarray, pcovs: np.ndarray):
    perrs = np.sqrt(np.diag(pcovs))
    perr_rels = abs(perrs / popts)
    for p, popt, perr_rel in zip(pnames, popts, perr_rels):
        print(f"{p:6s} = {popt:6.3f} ± {100*perr_rel:4.2f}%")

def sigmoid(x: float, x0: float, y0: float, sx: float, sy: float) -> float:
    return y0 + sy / (1 + np.exp(-(x - x0) / sx))

def get_sigmoid_fit(xs: list[float], ys: list[float]) -> tuple[np.ndarray, np.ndarray]:
    p0 = [np.median(xs), min(ys), max(xs) - min(xs), max(ys) - min(ys)]
    return opt.curve_fit(sigmoid, xs, ys, p0, method='dogbox')

def get_growth_rate_sigmoid(popts: np.ndarray, pcovs: np.ndarray) -> tuple[float, float]:
    return 1 / popts[2], np.sqrt(pcovs[2, 2]) / popts[2]**2

def exponential(x: float, y0: float, sx: float, sy: float) -> float:
    return y0 + sy * np.exp(x / sx)

def get_exponential_fit(xs: list[float], ys: list[float]) -> tuple[np.ndarray, np.ndarray]:
    p0 = [min(ys), max(xs) - min(xs), max(ys) - min(ys)]
    return opt.curve_fit(exponential, xs, ys, p0, method='dogbox')

def get_growth_rate_exponential(popts: np.ndarray, pcovs: np.ndarray) -> tuple[float, float]:
    return 1 / popts[1], np.sqrt(pcovs[1, 1]) / popts[1]**2

# Load Data

In [ ]:
path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/exact/B00.25-n512-cont/"

run_manager = bgk.RunManager(path)
params_record = run_manager.params_record
run_diagnostics = run_manager.run_diagnostics

size = run_diagnostics.domain_size
struct_radius = run_diagnostics.hole_radius

wholeSlice = bgk.DataSlice(slice(None, None), "")
centerSlice = bgk.DataSlice(slice(-struct_radius, struct_radius), "Central ")

run_diagnostics.print_params()
run_diagnostics.check_params()

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 201

videoMaker = bgk.VideoMaker(nframes, run_manager)

videoMaker.frame_manager.print_coverage()

In [ ]:
videoMaker.loadData(rp.ne)
videoMaker.setSlice(wholeSlice)

# Preprocessing

In [ ]:
rolling_window = 30
ts = moving_average(videoMaker.times, rolling_window)
ys = moving_average(videoMaker._getMeansAtOrigin(), rolling_window)

## Fit Sigmoid

In [ ]:
popts_sigmoid, pcovs_sigmoid = get_sigmoid_fit(ts, ys)

In [ ]:
plot(ts, [ys, "."], [lambda t: sigmoid(t, *popts_sigmoid), "-"])
print_errs(["t0", "y0", "st", "sy"], popts_sigmoid, pcovs_sigmoid)

## Fit Exponential

In [ ]:
tstart = 0
tstop = 30
growth_phase = slice(np.argmax(ts > tstart), np.argmax(ts > tstop))

ts2 = ts[growth_phase]
ys2 = ys[growth_phase]

In [ ]:
popts_exp, pcovs_exp = get_exponential_fit(ts2, ys2)

In [ ]:
print_errs(["y0", "st", "sy"], popts_exp, pcovs_exp)
plot(ts2, [ys2, "."], [lambda t: exponential(t, *popts_exp), "-"], [lambda t: -.004 + exponential(t - popts_sigmoid[0], *popts_sigmoid[1:]), "--"])

In [ ]:
growth_rate_sigmoid, growth_rate_sigmoid_err = get_growth_rate_sigmoid(popts_sigmoid, pcovs_sigmoid)
growth_rate_exp, growth_rate_exp_err = get_growth_rate_exponential(popts_exp, pcovs_exp)

print(f"growth rate sigmoid:     {growth_rate_sigmoid:.4f} ± {growth_rate_sigmoid_err:.4f}")
print(f"growth rate exponential: {growth_rate_exp:.4f} ± {growth_rate_exp_err:.4f}")